# Subgroups

Part of a [Recidivism Case Study](https://github.com/AllenDowney/RecidivismCaseStudy)

by [Allen Downey](https://allendowney.com)

[MIT License](https://opensource.org/licenses/MIT)

## Review

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data

Thank you to ProPublica and the authors of "Machine Bias" for making their data and analysis freely available.

[Their repository](https://github.com/propublica/compas-analysis) contains the data and analysis pipeline described in [this supplementary article](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm).

The terms of use for the data [are here](https://www.propublica.org/datastore/terms).  In compliance with those terms, I am not redistributing the data.

The following cell downloads the data file we'll use directly from their repository.

In [18]:
import os

if not os.path.exists('compas-scores-two-years.csv'):
    !wget https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv

The following cell read the data file:

In [19]:
cp = pd.read_csv("compas-scores-two-years.csv")
cp.shape

(7214, 53)

## Code

## The reference class problem

In the first notebook we replicated the analysis in the ProPublica article and computed various metrics of accuracy for white and black defendants.  We found that the predictive values (PPV and NPV) were similar for the two groups, but the error rates (FPR and FNR) were substantially different.

In the previous notebook, we ran the same analysis for male and female defendants and found the opposite pattern: the error rates are about the same, but the predictive values are different.

Neither scenario seems completely fair.  Comparing black and white defendants:

* Among non-recidivists, black defendants were more likely to be classified as high risk (false positive).

* Among recidivists, white defendants were more likely to be classified as low risk (false negative).

Comparing male and female defendants:

* Among defendants classified as high risk, female defendants were less likely to recidivate; that is, the test has lower PPV for women.

* Among defendants classified as low risk, male defendants were less likely to "survive"; that is, the test has lower NPV for men.

So it seems like we can't win.  If predictive values are the same for all groups, error rates are not (in general).  And if error rates are the same, predictive values are not.

Nevertheless, the designers of a test like COMPAS can make trade-offs among these kinds of errors.  And that raises two questions I would like to explore:

1. What kind of fairness is COMPAS designed to achieve?  Are they trying to make predictive value the same for all groups?  Or error rates?  Or some compromise?

2. What kind of fairness *should* a test like COMPAS achieve?

To explore the first question, I will compute accuracy metrics for defendants grouped by age, race, and sex, and for the intersections of these groups (explained below).

## Groups

The following function takes a DataFrame containing COMPAS data and a list of variables to group by.  It returns a table with one row for each group and one column for each accuracy metric.

In [58]:
def make_table(cp, group_vars):
    """Make a table of metrics for each subgroup.
    
    cp: DataFrame of COMPAS data
    group_vars: string or list of variable names to group by
    
    returns: DataFrame
    """
    # count the number of defendants in each group
    grouped = cp.groupby(group_vars)
    counts = grouped['id'].count()

    # make the table
    columns = ['FPR', 'FNR', 'PPV', 'NPV', 'Prevalence']
    table = pd.DataFrame(index=counts.index, 
                         columns=columns, 
                         dtype=float)

    # fill in the table
    for name, group in grouped:
        if len(group) < 50:
            continue
        m = make_matrix(group)
        metrics = compute_metrics(m)
        table.loc[name] = metrics['Percent']
    
    table['Count'] = counts
    return table

Here's the table for defendants grouped by race.

In [59]:
table1 = make_table(cp, 'race')
table1

,FPR,FNR,PPV,NPV,Prevalence,Count
race,,,,,,
African-American,44.846797,27.985271,62.971481,65.045992,51.433983,3696
Asian,NaN,NaN,NaN,NaN,NaN,32
Caucasian,23.454301,47.722567,59.133489,71.187500,39.364303,2454
Hispanic,21.481481,55.603448,54.210526,71.140940,36.420722,637
Native American,NaN,NaN,NaN,NaN,NaN,18
Other,14.754098,67.669173,54.430380,69.798658,35.278515,377


Some rows contain `NaN` values because I did not compute metrics for groups with fewer than 50 defendants.

`table1` confirms results from previous analysis and extends them to other groups:

* Predicive values are comparable for all racial groups, although PPV is highest for black defendants and NPV is lowest.

* Error rates are substantially different in different groups.  FPR is highest for black defendants, lower for white and Hispanic defendants, and lowest for defendants in other racial groups.  FNR is highest for Other and lowest for African-American.

The following table shows the breakdown by sex.

In [60]:
table2 = make_table(cp, 'sex')
table2

,FPR,FNR,PPV,NPV,Prevalence,Count
sex,,,,,,
Female,32.107023,39.156627,51.269036,75.746269,35.698925,1395
Male,32.420091,37.086814,63.536317,66.989977,47.310534,5819


Again, these results are consistent with previous analysis.  Comparing male and female defendants, the error rates are comparable, but the predictive values are substantially different.

Next we'll look at the breakdown by age, but first I'll recode the age categories so their ordering in the table is more logical:

In [61]:
cp['age_cat'].value_counts().index

Index(['25 - 45', 'Greater than 45', 'Less than 25'], dtype='object')

In [62]:
d = {'Less than 25':   '1 Younger than 25',
     '25 - 45':        '2 Between 25 - 45', 
     'Greater than 45':'3 Older than 45'}
cp['age_cat_recode'] = cp['age_cat'].replace(d)

Here's the breakdown by age group:

In [63]:
table3 = make_table(cp, 'age_cat_recode')
table3

,FPR,FNR,PPV,NPV,Prevalence,Count
age_cat_recode,,,,,,
1 Younger than 25,54.135338,26.041667,63.963964,57.547170,56.507521,1529
2 Between 25 - 45,33.378378,37.374272,61.486486,67.688787,45.972256,4109
3 Older than 45,16.790353,57.228916,54.060914,75.888325,31.598985,1576


This table shows some patterns we have seen before 

* In groups with high prevalence, FPR is relatively high and FNR relatively low.

* In groups with high prevalence, PPV is relatively hight and NPV relatively low.

But it is still not clear whether COMPAS is trying to achieve constant error rates, constant predictive values, or a compromise between the two.

It is also not clear what we can say about defendants in the intersections of these groups.  What are the metrics for a white male, or an older Hispanic defendant?

I'll compute these intersections in the next section.

## Intersections

In this section I'll compute metrics for defendants grouped by

* Race and sex

* Race and age

* Sex and age

* Race, sex, and age

Then we'll plot the results.

In [64]:
table4 = make_table(cp, ['race', 'sex'])
table4

FPR        FNR        PPV        NPV  \
race             sex                                                  
African-American Female  40.493827  29.959514  51.335312  76.507937   
                 Male    46.115108  27.690447  65.106151  62.054681   
Asian            Female        NaN        NaN        NaN        NaN   
                 Male          NaN        NaN        NaN        NaN   
Caucasian        Female  30.163043  43.216080  50.446429  74.927114   
                 Male    21.250000  48.891786  62.222222  70.167064   
Hispanic         Female  10.000000  72.727273  56.250000  72.413793   
                 Male    23.880597  52.763819  54.022989  70.833333   
Native American  Female        NaN        NaN        NaN        NaN   
                 Male          NaN        NaN        NaN        NaN   
Other            Female  11.538462  66.666667  45.454545  82.142857   
                 Male    15.625000  67.796610  55.882353  66.942149   

                         Prevalence  Count  
race             sex                        
African-American Female   37.883436    652  
                 Male     54.336399   3044  
Asian            Female         NaN      2  
                 Male           NaN     30  
Caucasian        Female   35.097002    567  
                 Male     40.646529   1887  
Hispanic         Female   32.038835    103  
                 Male     37.265918    534  
Native American  Female         NaN      4  
                 Male           NaN     14  
Other            Female   22.388060     67  
                 Male     38.064516    310

In [65]:
table5 = make_table(cp, ['race', 'age_cat_recode'])
table5

FPR        FNR        PPV  \
race             age_cat_recode                                       
African-American 1 Younger than 25  59.888579  23.172906  66.718266   
                 2 Between 25 - 45  44.095941  27.657658  62.685402   
                 3 Older than 45    31.818182  41.304348  54.655870   
Asian            1 Younger than 25        NaN        NaN        NaN   
                 2 Between 25 - 45        NaN        NaN        NaN   
                 3 Older than 45          NaN        NaN        NaN   
Caucasian        1 Younger than 25  48.743719  26.701571  59.071730   
                 2 Between 25 - 45  26.809651  46.996466  60.000000   
                 3 Older than 45     9.576427  68.899522  55.555556   
Hispanic         1 Younger than 25  47.540984  36.363636  59.154930   
                 2 Between 25 - 45  18.534483  59.259259  56.122449   
                 3 Older than 45    13.392857  80.645161  28.571429   
Native American  1 Younger than 25        NaN        NaN        NaN   
                 2 Between 25 - 45        NaN        NaN        NaN   
                 3 Older than 45          NaN        NaN        NaN   
Other            1 Younger than 25  45.238095  47.500000  52.500000   
                 2 Between 25 - 45  10.791367  73.239437  55.882353   
                 3 Older than 45     3.174603  86.363636  60.000000   

                                          NPV  Prevalence  Count  
race             age_cat_recode                                   
African-American 1 Younger than 25  52.554745   60.978261    920  
                 2 Between 25 - 45  66.374589   50.592525   2194  
                 3 Older than 45    71.641791   39.518900    582  
Asian            1 Younger than 25        NaN         NaN      7  
                 2 Between 25 - 45        NaN         NaN     14  
                 3 Older than 45          NaN         NaN     11  
Caucasian        1 Younger than 25  66.666667   48.974359    390  
                 2 Between 25 - 45  67.241379   43.140244   1312  
                 3 Older than 45    77.322835   27.792553    752  
Hispanic         1 Younger than 25  57.142857   51.968504    127  
                 2 Between 25 - 45  70.260223   36.784741    367  
                 3 Older than 45    79.508197   21.678322    143  
Native American  1 Younger than 25        NaN         NaN      3  
                 2 Between 25 - 45        NaN         NaN     12  
                 3 Older than 45          NaN         NaN      3  
Other            1 Younger than 25  54.761905   48.780488     82  
                 2 Between 25 - 45  70.454545   33.809524    210  
                 3 Older than 45    76.250000   25.882353     85

In [66]:
table6 = make_table(cp, ['sex', 'age_cat'])
table6

FPR        FNR        PPV        NPV  \
sex    age_cat                                                       
Female 25 - 45          29.411765  40.764331  56.193353  73.109244   
       Greater than 45  15.283843  66.197183  40.677966  80.497925   
       Less than 25     61.714286  17.699115  46.268657  77.011494   
Male   25 - 45          34.510712  36.698413  62.586315  66.179052   
       Greater than 45  17.196702  55.737705  56.417910  74.707758   
       Less than 25     51.428571  27.296937  68.421053  53.724605   

                        Prevalence  Count  
sex    age_cat                             
Female 25 - 45           38.909542    807  
       Greater than 45   23.666667    300  
       Less than 25      39.236111    288  
Male   25 - 45           47.698365   3302  
       Greater than 45   33.463950   1276  
       Less than 25      60.515713   1241

In [67]:
table7 = make_table(cp, ['race', 'sex', 'age_cat_recode'])
table7

FPR        FNR        PPV  \
race             sex    age_cat_recode                                       
African-American Female 1 Younger than 25  63.440860  19.736842  50.833333   
                        2 Between 25 - 45  36.475410  34.437086  52.659574   
                        3 Older than 45    23.529412  35.000000  44.827586   
                 Male   1 Younger than 25  58.646617  23.711340  70.342205   
                        2 Between 25 - 45  46.309524  26.590198  64.409881   
                        3 Older than 45    33.802817  41.904762  55.963303   
Asian            Female 2 Between 25 - 45        NaN        NaN        NaN   
                        3 Older than 45          NaN        NaN        NaN   
                 Male   1 Younger than 25        NaN        NaN        NaN   
                        2 Between 25 - 45        NaN        NaN        NaN   
                        3 Older than 45          NaN        NaN        NaN   
Caucasian        Female 1 Younger than 25  70.000000   3.703704  38.235294   
                        2 Between 25 - 45  29.213483  41.221374  59.689922   
                        3 Older than 45    13.076923  75.609756  37.037037   
                 Male   1 Younger than 25  39.568345  30.487805  67.455621   
                        2 Between 25 - 45  26.056338  48.735632  60.107817   
                        3 Older than 45     8.474576  67.261905  61.111111   
Hispanic         Female 1 Younger than 25        NaN        NaN        NaN   
                        2 Between 25 - 45   7.142857  80.952381  57.142857   
                        3 Older than 45          NaN        NaN        NaN   
                 Male   1 Younger than 25  52.941176  37.288136  57.812500   
                        2 Between 25 - 45  21.052632  55.263158  56.043956   
                        3 Older than 45    13.829787  76.923077  31.578947   
Native American  Female 2 Between 25 - 45        NaN        NaN        NaN   
                        3 Older than 45          NaN        NaN        NaN   
                 Male   1 Younger than 25        NaN        NaN        NaN   
                        2 Between 25 - 45        NaN        NaN        NaN   
                        3 Older than 45          NaN        NaN        NaN   
Other            Female 1 Younger than 25        NaN        NaN        NaN   
                        2 Between 25 - 45        NaN        NaN        NaN   
                        3 Older than 45          NaN        NaN        NaN   
                 Male   1 Younger than 25  46.666667  45.945946  58.823529   
                        2 Between 25 - 45  12.612613  75.806452  51.724138   
                        3 Older than 45     3.921569  84.210526  60.000000   

                                                 NPV  Prevalence  Count  
race             sex    age_cat_recode                                   
African-American Female 1 Younger than 25  69.387755   44.970414    169  
                        2 Between 25 - 45  74.879227   38.227848    395  
                        3 Older than 45    88.135593   22.727273     88  
                 Male   1 Younger than 25  48.888889   64.580559    751  
                        2 Between 25 - 45  63.881020   53.307393   1799  
                        3 Older than 45    68.115942   42.510121    494  
Asian            Female 2 Between 25 - 45        NaN         NaN      1  
                        3 Older than 45          NaN         NaN      1  
                 Male   1 Younger than 25        NaN         NaN      7  
                        2 Between 25 - 45        NaN         NaN     13  
                        3 Older than 45          NaN         NaN     10  
Caucasian        Female 1 Younger than 25  94.736842   31.034483     87  
                        2 Between 25 - 45  70.000000   42.394822    309  
                        3 Older than 45    78.472222   23.976608    171  
                 Male   1 Younger than 25  62.686567   5

These tables show how much these metrics vary between groups:

* 

In [68]:
tables = [table1, table2, table3, table4, table5, table6, table7]

all_groups = pd.concat(tables)

In [79]:
def min_max_metric(all_groups, metric):
    idx = all_groups[metric].idxmax()
    print(idx, all_groups[metric][idx])
    
    idx = all_groups[metric].idxmin()
    print(idx, all_groups[metric][idx])

Here are the groups with maximumn and minimum FPR:

In [81]:
min_max_metric(all_groups, 'FPR')

('Caucasian', 'Female', '1 Younger than 25') 70.0
('Other', '3 Older than 45') 3.1746031746031744


And here are the results for the other metrics:

In [85]:
min_max_metric(all_groups, 'FNR')

('Other', '3 Older than 45') 86.36363636363636
('Caucasian', 'Female', '1 Younger than 25') 3.7037037037037033


In [86]:
min_max_metric(all_groups, 'PPV')

('African-American', 'Male', '1 Younger than 25') 70.34220532319392
('Hispanic', '3 Older than 45') 28.57142857142857


In [87]:
min_max_metric(all_groups, 'NPV')

('Caucasian', 'Female', '1 Younger than 25') 94.73684210526315
('Other', 'Male', '1 Younger than 25') 48.484848484848484


In [88]:
min_max_metric(all_groups, 'Prevalence')

('African-American', 'Male', '1 Younger than 25') 64.58055925432757
('Hispanic', 'Male', '3 Older than 45') 21.666666666666668


Looking at these results, I have a few thoughts:

1. The differences between groups are big.  FNR, which ranges from about 4% to 86%, is particularly striking.

2. For some metrics the most extreme groups are relatively small groups, so that might not be meaningful.  Statistically it is easier for a small group to deviate from the overall averages.

3. It is not clear from these results how much of the variation between groups is due to differences in prevalance.  I'll explore this relationship in the next section.




## Reverse engineering



In [50]:
matrix_all = make_matrix(cp)
ppv, npv = predictive_value(matrix_all)
fpr, fnr = error_rates(matrix_all)

In [51]:
prevalences = np.linspace(20, 65, 11)

pred_pv = pd.DataFrame(columns=['ppv', 'npv'])

for prev in prevalences:
    m = constant_error_rates(fpr, fnr, prev)
    pred_pv.loc[prev] = predictive_value(m)
    
pred_pv

,ppv,npv
20.0,32.603329,87.856159
24.5,38.571828,84.787671
29.0,44.145209,81.577315
33.5,49.361450,78.215019
38.0,54.253815,74.689735
42.5,58.851558,70.989317
47.0,63.180516,67.100383
51.5,67.263589,63.008148
56.0,71.121148,58.696245
60.5,74.771374,54.146498


In [ ]:
prevalences = np.linspace(35, 55, 11)

pred_er = pd.DataFrame(columns=['FPR', 'FNR'])

for prev in prevalences:
    m = constant_predictive_value(ppv, npv, prev)
    pred_er.loc[prev] = error_rates(m)
    
pred_er

In [ ]:
def plot_cpv_model(pred_er):
    """Plot error rates with constant predictive values.
    
    pred_er: DataFrame of predicted error rates
    """
    pred_er['fpr'].plot(label='Predicted FPR', color='C2')
    pred_er['fnr'].plot(label='Predicted FNR', color='C4')
    decorate(xlabel='Prevalence', ylabel='Percent',
             title='Error rates, constant predictive value')

In [ ]:
def plot_cer_model(pred_pv):
    """Plot error rates with constant predictive values.
    
    pred_er: DataFrame of predicted error rates
    """
    pred_pv['ppv'].plot(label='Predicted PPV', color='C3')
    pred_pv['npv'].plot(label='Predicted NPV', color='C9')
    decorate(xlabel='Prevalence', ylabel='Percent',
             title='Predictive value, constant error rates')

Those are all the possible subgroups for these three variables.

Now we can see what the results look like.

The following function plots one data point per subgroup showing the given metric versus prevalence.

Groups with a small number of people are shown with lighter colors.


In [ ]:
def plot_table_var(table, var, color):
    """Plot one data point per row.
    
    table: DataFrame
    var: which metric to plot
    color: string
    """
    for _, row in table.iterrows():
        alpha = 0.8 if row['Count'] > 200 else 0.3

        plt.plot(row['Prevalence'], row[var],
                 'o', color=color, alpha=alpha)

In [ ]:
def plot_table_var(table, var, color):
    """Plot one data point per row.
    
    table: DataFrame
    var: which metric to plot
    color: string
    """
    plt.plot(table['Prevalence'], table[var],
                 'o', color=color, alpha=0.6)

Here's what the results look like for FPR.

In [ ]:
pred_er['FPR'].plot(label='Expected FPR, constant PV', color='C2')
plt.axhline(fpr, linestyle='dotted', color='gray',
            label='Expected FPR, constant ER')

for table in tables:
    plot_table_var(table, 'FPR', 'C2')
    
decorate(xlabel='Prevalence',
         ylabel='False positive rate',
         title='False positive rates by subgroup')
plt.legend();

In [ ]:
pred_er['FNR'].plot(label='Expected FNR, constant PV', color='C4')
plt.axhline(fnr, linestyle='dotted', color='gray',
            label='Expected FNR, constant ER')

for table in tables:
    plot_table_var(table, 'FNR', 'C4')
    
decorate(xlabel='Prevalence',
         ylabel='False negative rate',
         title='False negative rates by subgroup')
plt.legend();

In general, groups with higher prevalence have higher false positive rates, but the effect is less extreme than what we would expect from the CPV model.

Here are the results for positive predictive value.

In [ ]:
pred_pv['ppv'].plot(label='Expected PPV, constant FPR', color='C0')
plt.axhline(ppv, linestyle='dotted', color='gray',
            label='Expected PPV, constant PPV')

for table in tables:
    plot_table_var(table, 'PPV', 'C0')
    
decorate(xlabel='Prevalence',
         ylabel='Positive predictive value',
         title='Positive predictive value by subgroup')

plt.legend();

Groups with higher prevalence have higher PPV, but the effect is less extreme than we would expect from the CPV model.

Here are the results for false negative rate.

Groups with higher prevalence have lower FNR, but the effect is less extreme than we would expect from the CPV model.

Here are the results for negative predictive value.

In [ ]:
pred_pv['npv'].plot(label='Expected NPV, constant FPR', color='C1')
plt.axhline(npv, linestyle='dotted', color='gray',
            label='Expected NPV, constant NPV')

for table in tables:
    plot_table_var(table, 'NPV', 'C1')
    
decorate(xlabel='Prevalence',
         ylabel='Negative predictive value',
         title='Negative predictive value by subgroup')

plt.legend();

Groups with higher prevalence have lower NPV.  In this case, the effect is almost exactly what we would expect from the CPV model.

## IFPR given probability of recidivism

Imagine, temporarily, that we know the actual probability of recidivism for a given defendant, and we would like to know the probability that the defendant will turn out to be a false positive.

In order for that to happen, they would need to be classified as high risk, and then "survive" two years without being charged with another crime.

Given their probability of recidivism, $p$, we can figure out their correct risk score using the calibration curve.

But we should not assume that COMPAS always gives people the right risk score.  Instead, I'll assume it has a normal distribution of errors with mean 0 and an unknown standard deviation.

With these assumptions, we can compute the probability of being a false positive:

$P(FP ~|~ p) = \sum_{error} ~P(error) ~P(positive ~|~ error) ~P(survive)$

And that's what the following function computes.

In [ ]:
def individual_fpr(actual_prob_recid, cal, thresh, std_dev):
    """Compute an individual FPR given actual prob of recid.
    
    actual_prob_recid: actual probability of recidivism
    cal: calibration curve, map from score to prob_recid
    thresh: threshold between low and not low risk
    std_dev: standard deviation of the error function
    
    returns: individual FPR
    """
    # look up actual_prob_recid to get correct score
    correct_score = crossing(cal, actual_prob_recid)

    # make the error distribution
    error_dist = make_error_dist(std_dev)

    # loop through possible errors
    total_prob = 0
    for error, prob_error in error_dist.iteritems():
        # hypothetical score
        score = correct_score+error
        score = max(score, 1)
        score = min(score, 10)
        
        # probability of being classified 'not low' | error
        prob_positive = 1 if score > thresh else 0

        # probability of being a false positive | error
        prob_fp = prob_positive * (1-actual_prob_recid)
        
        total_prob += prob_error * prob_fp
    return total_prob

In [ ]:
def compute_error_rates(actual_prob_recid, cp, thresh):
    """Compute an individual FPR given actual prob of recid.
    
    actual_prob_recid: actual probability of recidivism
    cp: DataFrame of COMPAS data
    thresh: threshold between low and not low risk
    
    returns: individual FPR
    """
    recid = (cp['two_year_recid'] == 1)
    scores_recid = cp.loc[recid, 'decile_score']
    scores_norecid = cp.loc[~recid, 'decile_score']

    pmf_scores_recid = make_pmf(scores_recid)
    pmf_scores_norecid = make_pmf(scores_norecid)
    
    score_dist = (actual_prob_recid * pmf_scores_recid +
                 (1-actual_prob_recid) * pmf_scores_norecid)
    normalize(score_dist)
    
    positive = (score_dist.index > thresh)
    prob_positive = score_dist[positive].sum()
    
    prob_fp = prob_positive * (1-actual_prob_recid)
    prob_fn = (1-prob_positive) * actual_prob_recid
    
    return prob_fp, prob_fn

In [ ]:
compute_error_rates(0.3, cp, 4.5)

In [ ]:
compute_error_rates(0.5, cp, 4.5)

In [ ]:
compute_error_rates(0.7, cp, 4.5)

In [ ]:
def error_rates_vs_prob_recid(cp, thresh):
    """Computes FPR for a range of prob_recid.
    
    cp:
    thresh: threshold between low and not low risk
    
    returns: Series
    """
    index = np.linspace(0, 1, 11)
    columns = ['FPR', 'FNR']
    er = pd.DataFrame(index=index, columns=columns, dtype=float)
    
    for prob_recid in index:
        fpr, fnr = compute_error_rates(prob_recid, cp, thresh)
        er.loc[prob_recid] = fpr, fnr
        
    return er

In [ ]:
er_all = error_rates_vs_prob_recid(cp, thresh=4.5)
er_all

In [ ]:
er_all.plot(color=['C2', 'C4'])

decorate(xlabel='Actual probability of recidivism',
         ylabel='Probability',
         title='FPR and FNR vs probability of recidivism')
plt.legend();

In [ ]:
white = (cp.race=='Caucasian')
er_white = error_rates_vs_prob_recid(cp[white], thresh=4.5)
er_white

In [ ]:
black = (cp.race=='African-American')
er_black = error_rates_vs_prob_recid(cp[black], thresh=4.5)
er_black

In [ ]:
er_all['FPR'].plot(color='gray', label='all')
er_black['FPR'].plot(label='Black defendants')
er_white['FPR'].plot(label='White defendants')

plt.legend();

In [ ]:
er_all['FNR'].plot(color='gray', label='all')
er_black['FNR'].plot(label='Black defendants')
er_white['FNR'].plot(label='White defendants')

plt.legend();

### What would it take?

In this section I explore what it would take to make a test with the same false positive rate for all groups.

In [ ]:
def fpr_thresh(df, thresh):
    df = df.copy()
    df['high'] = df['decile_score'] >= thresh
    matrix_all = crosstab(df, 'two_year_recid', 'high')
    fpr, fnr = error_rates(matrix_all)
    return fpr

In [ ]:
fpr_thresh(cp, 5)

In [ ]:
fpr_thresh(black, 5)

In [ ]:
fpr_thresh(white, 5)

In [ ]:
def sweep_thresh(df):
    threshes = range(2,10)
    sweep = pd.Series(index=threshes, dtype=float)
    for thresh in threshes:
        sweep[thresh] = fpr_thresh(df, thresh)
        
    return sweep

In [ ]:
plt.axhline(32.25, color='gray')
sweep_thresh(cp).plot(label='All')
sweep_thresh(black).plot(label='Black')
sweep_thresh(white).plot(label='White')
decorate(xlabel='Threshold',
         ylabel='False positive rate')

In [ ]:
def find_threshold(group, fpr):
    series = sweep_thresh(group)
    xs = crossing(series.dropna(), fpr)
    return xs

In [ ]:
all_thresh = find_threshold(cp, 32.35)
all_thresh

In [ ]:
black_thresh = find_threshold(black, 32.35)
black_thresh

In [ ]:
white_thresh = find_threshold(white, 32.35)
white_thresh

In [ ]:
interpolate(calibration_curve(cp), all_thresh)

In [ ]:
interpolate(calibration_curve(cp), black_thresh)

In [ ]:
interpolate(calibration_curve(black), black_thresh)

In [ ]:
interpolate(calibration_curve(cp), white_thresh)

In [ ]:
interpolate(calibration_curve(white), white_thresh)

In [ ]:
black_male = black[black.sex=='Male']
black_male.shape

In [ ]:
black_female = black[black.sex=='Female']
black_female.shape

In [ ]:
old_black_female = black_female[black_female.age_cat=='Greater than 45']
old_black_female.shape

In [ ]:
old_white_female = cp[(cp.age_cat=='Greater than 45') &
                      (cp.sex=='Female') &
                      (cp.race=='Caucasian')]
old_white_female.shape

In [ ]:
young_black_male = cp[(cp.age_cat=='Less than 25') &
                      (cp.sex=='Male') &
                      (cp.race=='African-American')]
young_black_male.shape

In [ ]:
fpr_thresh(cp, 5)

In [ ]:
fpr_thresh(black, 5)

In [ ]:
fpr_thresh(black_female, 5)

In [ ]:
fpr_thresh(old_black_female, 5)

In [ ]:
fpr_thresh(black_male, 5)

In [ ]:
fpr_thresh(young_black_male, 5)

In [ ]:
plt.axhline(32.25, color='gray')
sweep_thresh(black).plot(label='Black', color='gray')
sweep_thresh(black_male).plot(label='Black male')
sweep_thresh(young_black_male).plot(label='Young black male')

decorate(xlabel='Threshold',
         ylabel='False positive rate')

In [ ]:
plt.axhline(32.25, color='gray')
sweep_thresh(black).plot(label='Black', color='gray')
sweep_thresh(black_female).plot(label='Black female')
sweep_thresh(old_black_female).plot(label='Old black female')

decorate(xlabel='Threshold',
         ylabel='False positive rate')

In [ ]:
ybm_thresh = find_threshold(young_black_male, 32.35)

In [ ]:
obf_thresh = find_threshold(old_black_female, 32.35)

In [ ]:
interpolate(calibration_curve(cp), ybm_thresh)

In [ ]:
interpolate(calibration_curve(cp), obf_thresh)

## Dead Ends

In [ ]:
cp['recid'] = (cp['two_year_recid'] == 1)

In [ ]:
index = 'decile_score'
columns = 'race', 'recid'
table = cp.pivot_table(index=index, columns=columns, 
                       values='id', aggfunc='count', fill_value=0)
table

In [ ]:
for column in table.columns:
    normalize(table[column])
    
table

In [ ]:
table['Caucasian', True].cumsum().plot()
table['African-American', True].cumsum().plot()

In [ ]:
table['Caucasian', False].cumsum().plot()
table['African-American', False].cumsum().plot()

In [ ]:
cal_all = calibration_curve(cp)
cal_all.plot()
decorate(ylabel='Probability of recidivism')

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
def interpolate(series, value, **options):
    """Evaluate a function at a value.
    
    series: Series
    value: number
    options: passed to interp1d (default is linear interp)
    
    returns: number
    """
    interp = interp1d(series.index, series.values, **options)
    return interp(value)

In [ ]:
interpolate(cal_all, 3.5)

In [ ]:
interpolate(cal_all, 6.5)

In [ ]:
cp['est_prob_recid'] = interpolate(cal_all, cp['decile_score'])

In [ ]:
def make_pmf(series):
    """Make a PMF."""
    counts = series.value_counts(sort=False).sort_index()
    normalize(counts)
    return counts

def normalize(series):
    series /= series.sum()

def plot_pmf(pmf, **options):
    """Plot a PMF."""
    plt.plot(pmf.index, pmf.values, **options)

In [ ]:
def make_cdf(series):
    """Make a CDF."""
    return make_pmf(series).cumsum()

def plot_cdf(cdf, **options):
    """Plot a CDF as a step function."""
    plt.step(cdf.index, cdf.values, where='post', **options)

In [ ]:
# scores for recidivists and non
scores_recid = cp.loc[recid, 'decile_score']
scores_norecid = cp.loc[~recid, 'decile_score']

In [ ]:
pmf_scores_recid = make_pmf(scores_recid)
pmf_scores_norecid = make_pmf(scores_norecid)

In [ ]:
plot_pmf(pmf_scores_recid)
plot_pmf(pmf_scores_norecid)

In [ ]:
# implied probabilities for recidivists and non
probs_recid = cp.loc[recid, 'prob_recid']
probs_norecid = cp.loc[~recid, 'prob_recid']

In [ ]:
# distributions of implied probabilities for recivists and non
pmf_probs_recid = make_pmf(probs_recid)
pmf_probs_norecid = make_pmf(probs_norecid)

In [ ]:
plot_pmf(pmf_probs_recid)
plot_pmf(pmf_probs_norecid)

In [ ]:
# distributions of implied probabilities for recivists and non
cdf_probs_recid = make_cdf(probs_recid)
cdf_probs_norecid = make_cdf(probs_norecid)

In [ ]:
plot_cdf(cdf_probs_recid)
plot_cdf(cdf_probs_norecid)

In [ ]:
import scipy.stats

def estimate_beta(seq):
    """Estimate the parameters of a beta distribution.
    
    See https://en.wikipedia.org/wiki/Beta_distribution#Two_unknown_parameters
    
    seq: sequence of probabilities
    
    returns: scipy.stats.beta object
    """
    xbar = seq.mean()
    vbar = seq.var()
    assert vbar < xbar * (1-xbar)
    
    product = xbar * (1-xbar) / vbar - 1
    a = xbar * product
    b = (1-xbar) * product

    return scipy.stats.beta(a, b)

In [ ]:
dist_probs_recid = estimate_beta(probs_recid)
dist_probs_recid.args

In [ ]:
dist_probs_norecid = estimate_beta(probs_norecid)
dist_probs_norecid.args

In [ ]:
qs = np.linspace(0, 1, 21)
ps = dist_probs_recid.cdf(qs)
cdf_probs_recid_model = pd.Series(ps, index=qs)

ps = dist_probs_norecid.cdf(qs)
cdf_probs_norecid_model = pd.Series(ps, index=qs)

In [ ]:
cdf_probs_recid_model.plot(color='C0', alpha=0.4, label='recid model')
plot_cdf(cdf_probs_recid, label='recid data')

cdf_probs_norecid_model.plot(color='C1', alpha=0.4, label='norecid model')
plot_cdf(cdf_probs_norecid, label='norecid data')

decorate(xlabel='Probability of recidivism',
         ylabel='CDF',
         title='Distribution of probability')
plt.legend();

In [ ]:
n = len(cp)
prob_recid_sample = dist_probs_recid.rvs(n)
prob_norecid_sample = dist_probs_norecid.rvs(n)
flip = (np.random.random(n) < cp['prob_recid'])

In [ ]:
qs = np.linspace(0, 1, 21)
ps = dist_probs_recid.pdf(qs)
pdf_probs_recid_model = pd.Series(ps, index=qs)
pdf_probs_recid_model /= pdf_probs_recid_model.sum()

ps = dist_probs_norecid.pdf(qs)
pdf_probs_norecid_model = pd.Series(ps, index=qs)
pdf_probs_norecid_model /= pdf_probs_norecid_model.sum()

In [ ]:
plot_pmf(pmf_probs_recid)
plot_pmf(pmf_probs_norecid)

pdf_probs_recid_model.plot(color='C0', alpha=0.4, label='recid model')

pdf_probs_norecid_model.plot(color='C1', alpha=0.4, label='norecid model')

decorate(xlabel='Probability of recidivism',
         ylabel='PDF',
         title='Distribution of probability')
plt.legend();

In [ ]:
def crossing(series, value, **options):
    """Find where a function crosses a value.
    
    series: Series
    value: number
    options: passed to interp1d (default is linear interp)
    
    returns: number
    """
    interp = interp1d(series.values, series.index, **options)
    return interp(value)

In [ ]:
crossing(cal_all, 0.4)

In [ ]:
crossing(cal_all, 0.6)

In [ ]:
def convert_probs_to_scores(pmf):

    options = dict(bounds_error=False, fill_value=(1, 10))

    ps = pmf.to_numpy()
    index = pmf.index

    qs = crossing(cal_all, index, **options)

    return pd.Series(ps, index=qs)

In [ ]:
pdf_scores_recid_model = convert_probs_to_scores(pdf_recid_model)
pdf_scores_norecid_model = convert_probs_to_scores(pdf_norecid_model)

In [ ]:
pmf_scores_recid.plot(label='scores recid')
pmf_scores_norecid.plot(label='scores norecid')

pdf_scores_recid_model.plot(color='C0', alpha=0.4, label='recid model')

pdf_scores_norecid_model.plot(color='C1', alpha=0.4, label='norecid model')

decorate(xlabel='Risk score',
         ylabel='PDF',
         title='Distribution of risk scores')
plt.legend();

In [ ]:
actual_prob_recid = 0.5
prob_dist = (actual_prob_recid * pdf_probs_recid_model +
            (1-actual_prob_recid) * pdf_probs_norecid_model)
normalize(prob_dist)

In [ ]:
prob_dist.plot()

pdf_probs_recid_model.plot(color='C0', alpha=0.4, label='recid model')

pdf_probs_norecid_model.plot(color='C1', alpha=0.4, label='norecid model')

decorate(xlabel='Probability of recidivism',
         ylabel='PDF',
         title='Distribution of probability')
plt.legend();

In [ ]:
actual_prob_recid = 0.5
score_dist = (actual_prob_recid * pdf_scores_recid_model +
            (1-actual_prob_recid) * pdf_scores_norecid_model)
normalize(score_dist)

In [ ]:
score_dist.plot()

pdf_scores_recid_model.plot(color='C0', alpha=0.4, label='recid model')

pdf_scores_norecid_model.plot(color='C1', alpha=0.4, label='norecid model')

decorate(xlabel='Risk score',
         ylabel='PDF',
         title='Distribution of risk scores')
plt.legend();

In [ ]:
def make_error_dist(std_dev):
    """Make a discrete Gaussian distribution.
    
    std_dev: standard deviation
    
    returns: Series that maps errors to probabilities
    """
    errors = np.linspace(-4, 4, 21)
    prob_error = np.exp(-(errors/std_dev)**2)
    prob_error /= np.sum(prob_error)
    error_dist = pd.Series(prob_error, index=errors)
    return error_dist

In [ ]:
error_dist = make_error_dist(std_dev=2)
error_dist.plot(label='')
decorate(xlabel='Error (score)',
         ylabel='Probability')